# Proyecto Modulo 3


<a href="https://colab.research.google.com/github/ArturoG1z/Proyecto-Modulo-3-CyAD/blob/main/Notebook/proyecto_modulo_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

***PENDIENTE:*** Agregar descripción pequeña del problema del dataset y del objetivo


In [ ]:
import pandas as pd
import cufflinks as cf
# from IPython.display import display
cf.set_config_file(sharing='public', theme='pearl', offline=True)
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# ruta = '/content/drive/MyDrive/Colab Notebooks/PROYECTO/data set con color.xlsx'
# ruta = './data set Proyecto.xlsx'
ruta = './SEDUCA-2023B.xlsx'
df_original = pd.read_excel(ruta)

In [ ]:
df_original.head()

In [ ]:
df_original.shape

## Limpieza de los datos

### Eliminar columnas innecesarias

In [ ]:
# remove last 6 columns and first column
df_original = df_original.iloc[:, :-6]
df_original = df_original.iloc[:, 1:]

In [ ]:
df_original.dtypes

In [ ]:
quitar = [
  'Asesor',
  'Bloque',
  'Grupo',
  'Porcentaje-Eficiencia-Terminal',
]

#print(columnas)
df = df_original.drop(quitar, axis=1)

# Eliminando el ID de la materia en la columna 'Comunidad'
df['Comunidad'] = df['Comunidad'].str[5:]
df.head()

### Separar columnas compuestas

In [ ]:
# Reemplaza los valores NaN por 0
df.fillna(0, inplace=True)

In [ ]:
''' 
CONVERT TO INT
1p_total_actividades_entregadas
1p_total_actividades_entregadas
1p_total_actividades_calificadas_tiempo
2p_total_actividades_asignadas
2p_total_actividades_evaluativas
2p_total_actividades_calificadas
2p_total_actividades_entregadas
2p_total_actividades_calificadas_tiempo
2p_porcentaje_aprobados
total_actividades_calificadas
trabajos_por_calificar
Aprobados 
'''
columnas = [
  '1p_total_actividades_entregadas',
  '1p_total_actividades_entregadas',
  '1p_total_actividades_calificadas_tiempo',
  '2p_total_actividades_asignadas',
  '2p_total_actividades_evaluativas',
  '2p_total_actividades_calificadas',
  '2p_total_actividades_entregadas',
  '2p_total_actividades_calificadas_tiempo',
  '2p_porcentaje_aprobados',
  'total_actividades_calificadas',
  'trabajos_por_calificar',
  'Aprobados',
]

for col in columnas:
  df[col] = df[col].astype(int)

In [ ]:
df.dtypes

In [ ]:
# ver rango de valores de cada columna avoiding '<=' not supported between instances of 'str' and 'int' if its object and 
for col in df.columns:
  if df[col].dtype == 'object':
    print(col, df[col].unique())
  else:
    print(col, df[col].min(), df[col].max())

In [ ]:
# ver valores de 1p_porcentaje_evidencias_calificadas_tiempo mayores a 1
df[df['1p_porcentaje_evidencias_calificadas_tiempo'] > 1]
# show this rows putting the column at first place on a new dataframe
df[df['1p_porcentaje_evidencias_calificadas_tiempo'] > 1][['1p_porcentaje_evidencias_calificadas_tiempo'] + [col for col in df.columns if col != '1p_porcentaje_evidencias_calificadas_tiempo']]

In [ ]:
# Dado que  tiene valores mayores a 1, se procede a dividir por 100 dichos valores que son mayores a 1 (df['1p_porcentaje_evidencias_calificadas_tiempo'] > 1)
df['1p_porcentaje_evidencias_calificadas_tiempo'] = df['1p_porcentaje_evidencias_calificadas_tiempo'].apply(lambda x: x / 100 if x > 1 else x)

In [ ]:
''' 1er_P_Ponderación-semanas
2oP_Ponderación-semanas '''
# if is not like this n/n like '100/2' or is int then convert to 0/0 
df['1er_P_Ponderación-semanas'] = df['1er_P_Ponderación-semanas'].apply(lambda x: '0\\0' if type(x) == int or '/' not in x else x)
df['2oP_Ponderación-semanas'] = df['2oP_Ponderación-semanas'].apply(lambda x: '0\\0' if type(x) == int or '/' not in x else x)


In [ ]:
# Separando la columna 1er_P_Ponderación-semanas en : 1erPonderacion Semana_1erP splited by '\' and convert to int (nP_ponderacion y nP_semana) n es el parcial colocarlas en la misma posicion donde estaba la columna original

df[['1p_Ponderacion', '1p_semana']] = df['1er_P_Ponderación-semanas'].str.split('\\', expand=True)
df['1p_Ponderacion'] = df['1p_Ponderacion'].astype(int)
df['1p_semana'] = df['1p_semana'].astype(int)
df = df.drop('1er_P_Ponderación-semanas', axis=1)

# Separando la columna 2oP_Ponderación-semanas
df[['2p_Ponderacion', '2p_semana']] = df['2oP_Ponderación-semanas'].str.split('\\', expand=True)
df['2p_Ponderacion'] = df['2p_Ponderacion'].astype(int)
df['2p_semana'] = df['2p_semana'].astype(int)
df = df.drop('2oP_Ponderación-semanas', axis=1)


df.tail()
#df.shape

In [ ]:

# Cuenta cuántos 0 hay en la columna
numero_ceros_1er = df['1p_porcentaje_aprobados'].value_counts()[0]
numero_ceros_2do = df['2p_porcentaje_aprobados'].value_counts()[0]

# Imprime el número de 0
print(numero_ceros_1er)

# Imprime el número de 0
print(numero_ceros_2do)


### Obtener nuevas columnas calculadas

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
# get how many 1er P_Actividades asignadas are diferent than 1er P_Actividades evaluativas
# diferent_values = (numerical_df['1p_total_actividades_asignadas'] != numerical_df['1p_total_actividades_evaluativas']).sum()
# print(diferent_values)

# total de actividades asignadas = Alumnos * 1er P_Actividades asignadas 
# porcentaje as float64
df['1p_total_actividades_asignadas'] = df['alumnos'] * df['1p_total_actividades_asignadas']
df['2p_total_actividades_asignadas'] = df['alumnos'] * df['2p_total_actividades_asignadas']
df['total_actividades_asignadas'] = df['1p_total_actividades_asignadas'] + df['2p_total_actividades_asignadas']

In [ ]:
# porcentaje de actividades entregadas = 1p_total_actividades_entregadas / 1p_total_actividades_asignadas
df['1p_porcentaje_actividades_entregadas'] = (df['1p_total_actividades_entregadas'] / df['1p_total_actividades_asignadas']) * 100
df['2p_porcentaje_actividades_entregadas'] = (df['2p_total_actividades_entregadas'] / df['2p_total_actividades_asignadas']) * 100
df['porcentaje_actividades_entregadas'] = ((df['1p_total_actividades_entregadas'] + df['2p_total_actividades_entregadas']) / df['total_actividades_asignadas']) * 100

# change nan values to 0
df.fillna(0, inplace=True)

# replace inf values to 0
df.replace([np.inf, -np.inf], 0, inplace=True)

# convert each column to dtype float64
df['1p_porcentaje_actividades_entregadas'] = df['1p_porcentaje_actividades_entregadas'].astype(float)
df['2p_porcentaje_actividades_entregadas'] = df['2p_porcentaje_actividades_entregadas'].astype(float)
df['porcentaje_actividades_entregadas'] = df['porcentaje_actividades_entregadas'].astype(float)

In [ ]:
df.head()

### Filas con valores nulos

In [ ]:
# search for nan values in numerical_df
nan_values = df.isna().sum()
print(nan_values)

In [ ]:
# search for 0 values in numerical_df
zero_values = (df == 0).sum()
print(zero_values)

In [ ]:
# numero de registros que cumplen con la condicion de que todas las columnas que se pasan como parametro son 0
def count_zeros(df, columns):
  return (df[columns] == 0).all(axis=1).sum()


In [ ]:
# num of rows with 0 in each column 1p_total_actividades_asignadas and 1p_porcentaje_aprobados
count_zeros(df, ['1p_total_actividades_asignadas', '1p_porcentaje_aprobados', '2p_total_actividades_asignadas', '2p_porcentaje_aprobados'])

In [ ]:
relacion_datos_vacios = pd.DataFrame({
  '1p_con_actividades_asignadas_2p_sin_actividades_asignadas': df[(df['1p_total_actividades_asignadas'] == 0) & (df['2p_total_actividades_asignadas'] > 0)].shape[0],
  '1p_sin_actividades_asignadas_2p_con_actividades_asignadas': df[(df['2p_total_actividades_asignadas'] == 0) & (df['1p_total_actividades_asignadas'] > 0)].shape[0],
  '1p_sin_porcentaje_aprobados_2p_con_porcentaje_aprobados': df[(df['1p_porcentaje_aprobados'] == 0) & (df['2p_porcentaje_aprobados'] > 0)].shape[0],
  '1p_con_porcentaje_aprobados_2p_sin_porcentaje_aprobados': df[(df['2p_porcentaje_aprobados'] == 0) & (df['1p_porcentaje_aprobados'] > 0)].shape[0],
}, index=[0])

display(relacion_datos_vacios)


In [ ]:
((df['1p_total_actividades_asignadas'] == 0) & (df['1p_porcentaje_aprobados'] == 0) & (df['2p_total_actividades_asignadas'] == 0) & (df['2p_porcentaje_aprobados'] == 0)).sum()

In [ ]:
# remove rows with 0 in ['1p_total_actividades_asignadas', '1p_porcentaje_aprobados', '2p_total_actividades_asignadas', '2p_porcentaje_aprobados']
df = df[~((df['1p_total_actividades_asignadas'] == 0) & (df['1p_porcentaje_aprobados'] == 0) & (df['2p_total_actividades_asignadas'] == 0) & (df['2p_porcentaje_aprobados'] == 0))]

In [ ]:
df.shape

## Entrenar modelo


In [ ]:
# Split df into categorical df and numerical df first 4 columns are categorical and the rest are numerical
categorical_df = df.iloc[:, :4]
numerical_df = df.iloc[:, 4:]

In [ ]:
categorical_features = categorical_df.columns
numerical_features = numerical_df.columns

In [ ]:
import plotly.express as px

# Standardize the data to have a mean of ~0 and a variance of 1
X_std = StandardScaler().fit_transform(numerical_df)

# Create a PCA instance: pca
# pca = PCA(n_components=3)
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X_std)

# Save components to a DataFrame
# PCA_components = pd.DataFrame(principalComponents, columns=['pca1', 'pca2', 'pca3'])
PCA_components = pd.DataFrame(principalComponents, columns=['pca1', 'pca2'])

# df.head()

# Concatenate the DataFrames
df_final = pd.concat([PCA_components, df], axis=1)


# using  dendrogram, linkage, fcluster 
# Calculate the linkage: mergings
mergings = linkage(X_std, 'ward')

# Plot the dendrogram, using varieties as labels
plt.figure(figsize=(20, 10))
dendrogram(
  mergings,
  leaf_rotation=90,
  leaf_font_size=6,
) 
plt.show()

In [ ]:
clusters = fcluster(mergings, t=70, criterion='distance')

In [ ]:

# Create a DataFrame with labels and species as columns: df
df_final['cluster'] = 'Cluster ' + pd.Series(clusters).astype(str)
df_final.head()

# See cluster with pca

# fig = px.scatter_3d(df_final, x='pca1', y='pca2', z='pca3', color='cluster')
fig = px.scatter(df_final, x='pca1', y='pca2', color='cluster')
# increase the graph size
fig.update_layout(
  title='Clusters',
  scene=dict(
      xaxis=dict(title='PCA 1'),
      yaxis=dict(title='PCA 2'),
      # zaxis=dict(title='PCA 3')
  ),
  margin=dict(l=0, r=0, b=0, t=0),
  height=800,
  width=800
)
fig.show()

In [ ]:
df_final.dtypes

In [ ]:
# now with df original wee have 3 clusters to interpret the data make 3 dataframes with each cluster  (df_final['cluster']) and see the mean of each cluster
df_by_cluster = df_final.groupby("cluster")
df_by_cluster

In [ ]:
# display total of each cluster
for name, group in df_by_cluster:
  print(name, group.shape)

In [ ]:

# wee have 26 columns that are numerical from numerical_df

fig = px.scatter(df_final, x='total_actividades_asignadas', y='porcentaje_actividades_entregadas', color='cluster')
fig.show()
for col in numerical_features:
  fig = px.box(df_final, x='cluster', y=col)
  # add the mean of each cluster for the column
  fig.add_scatter(x=df_by_cluster[col].mean().index, y=df_by_cluster[col].mean().values, mode='markers', marker=dict(size=10, color='black'))
  fig.show()
  # imprime un grafico de barras con la media de cada columna por cluster en un solo grafico
  # fig = px.bar(df_by_cluster[col].mean().reset_index(), x='cluster', y=col)
  # fig.show()
  
  # y un histograma de la columna por cluster
  # fig = px.histogram(df_final, x=col, color='cluster')
  # fig.show()
  



In [ ]:
descriptions = {}

# get statistics of each cluster
for cluster, data in df_by_cluster:
  print(cluster)
  ## see graphics of each cluster
  """ Con los siguientes datos de un data set sobre rendimiento de grupos de universidad en una plataforma educativa virtual

  alumnos
  total_actividades_asignadas
  porcentaje_actividades_entregadas
  1p_porcentaje_actividades_entregadas
  2p_porcentaje_actividades_entregadas

  1p_total_actividades_calificadas
  1p_porcentaje_evidencias_calificadas_tiempo
  1p_total_actividades_calificadas_tiempo

  2p_total_actividades_calificadas
  2p_porcentaje_evidencias_calificadas_tiempo
  2p_total_actividades_calificadas_tiempo


  1p_porcentaje_aprobados
  2p_porcentaje_aprobados

  correos_enviados
  correos_recibidos

  con los 3 clusters obtenidos que etiquetan a estos datos, como obtener grafico de barras de 
  """
  description = data.describe()
  descriptions[cluster] = description
  display(description)
  print('\n\n')
  # correlation matrix only with numerical_features
  
  cluster_numerical_df = data[numerical_features]
  # correlation matrix
  correlation_matrix = cluster_numerical_df.corr()

  # plot correlation matrix with plotly
  fig = px.imshow(correlation_matrix, title=f'Correlation matrix for cluster {cluster}')
  # increase the graph size
  fig.update_layout(
    title=f'Correlation matrix for cluster {cluster}',
    margin=dict(l=0, r=0, b=0, t=0),
    height=1400,
    width=1400
  )
  fig.show()

  


In [ ]:
# pass to excel the description of each cluster
# description = data.describe()
# descriptions[cluster] = description


with pd.ExcelWriter("cluster_descriptions_hierarichal.xlsx") as writer:
  for cluster, description in descriptions.items():
    description.to_excel(writer, sheet_name=cluster)

In [ ]:
def scatter_by_category(df, x, y, category, title):
  fig = px.scatter(df, x=x, y=y, color=category, title=title)
  fig.show()

def count_by_category(df, category, title):
  fig = px.histogram(df, x=category, title=title)
  fig.update_layout(
    height=1200
  )
  fig.show()

In [ ]:
# for each cluster see a scatter plot of the 2 first principal components and labaled by categorical features

for cluster, data in df_by_cluster:
  scatter_by_category(data, 'pca1', 'pca2', 'Comunidad', f'Comunidad {cluster}')
  count_by_category(data, 'Comunidad', f'Comunidad {cluster}')

for cluster, data in df_by_cluster:
  scatter_by_category(data, 'pca1', 'pca2', 'carrera', f'Carrera {cluster}')
  count_by_category(data, 'carrera', f'Carrera {cluster}')

for cluster, data in df_by_cluster:
  scatter_by_category(data, 'pca1', 'pca2', 'Espacio_Académico', f'Espacio Académico {cluster}')
  count_by_category(data, 'Espacio_Académico', f'Espacio Académico {cluster}')